# Global Planning through CARLA API
In this notebook I will 

In [1]:
import sys
import os
import glob
# sys.path.append(glob.glob('C:/CARLA/WindowsNoEditor/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
#         sys.version_info.major,
#         sys.version_info.minor,
#         'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm

import carla



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
''' Connecting to Carla and fetching blue-print of the map'''

# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\Syedm\anaconda3\envs\carla\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Syedm\AppData\Local\Temp\ipykernel_30192\917811145.py", line 6, in <module>
    world = client.get_world()
RuntimeError: time-out of 20000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Syedm\anaconda3\envs\carla\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Syedm\anaconda3\envs\carla\lib\site-packages\IPython\core\ultratb.py",

TypeError: object of type 'NoneType' has no len()

In [62]:
start_point =  random.choice(spawn_points)
end_point = random.choice(spawn_points)


In [63]:
''' Trying to import the Global and Local '''
# sys.path.append(("C:/CARLA/WindowsNoEditor/PythonAPI/carla"))
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [64]:
''' Big Bren momment:
I found the navigation library implemented in CARLA: 
Need to further look into the functionality of:
1. Global Planner
2. Local Planner
3. Behaviour agent
From the looks of it behaviour agent uses the parent class of Basic Agent.
In any case Lets try to use the method 
_tailgating(elf, waypoint, vehicle_list): Used this script https://carla.readthedocs.io/en/0.9.12/adv_agents/
Read up on the link
'''

# To import a basic agent
from agents.navigation.basic_agent import BasicAgent

# To import a behavior agent
from agents.navigation.behavior_agent import BehaviorAgent

In [65]:
vehicle = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle, start_point) # use the .try_spawn_actor method always since sometimes the actor may collide at that spawn point

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)



In [66]:
start_point = start_point.location
end_point = end_point.location

In [67]:
'''Tracing a route between the two specified point '''

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
route =  grp.trace_route(start_point, end_point)

''' This is just to visualize the route'''
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)
    
print("Done")

Done


In [68]:
# To start a basic agent
agent = BasicAgent(vehicle)

# To start a behavior agent with an aggressive profile
agent = BehaviorAgent(vehicle, behavior='aggressive')
agent.set_destination(end_point, start_point)

In [69]:
while True:
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break

    vehicle.apply_control(agent.run_step())

The target has been reached, stopping the simulation


In [70]:
''' Destroy everything'''
try:
    car_lst = world.get_actors().filter('vehicle.*')
    print(car_lst)
    if car_lst:
        for car in car_lst:
            car.destroy()
except:
    print("Nope doesnt work")

[Actor(id=36, type=vehicle.lincoln.mkz_2020)]


Okay so lets make some notes:
1. Right now we arent doing anything exceptional since we are just using the API provided by CARLA.
2. However, if we want to conduct some research on Path planning we can leverage the BASIC Agent and behaviour agent script (This will implement the behavioural modelling on its on).
   1. The thing that we would have to do it figure outr how the global planner and local planner interacts with the behaviour classes. Then we can just change the given planner with our own
3. Try to see if I can change the perception side of things and make it more realistic


So immediate plan following this is:
1. Make a system design model how the navigation library seemingly works
2. check out the 

In [78]:
  # Toggle all buildings off
world.load_map_layer(carla.MapLayer.Buildings)